# 라이브러리 import

In [17]:
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
import re
from tqdm import tqdm
from selenium.common.exceptions import NoSuchElementException


# 함수 정의

In [19]:
def get_url_list():
    with open("../data/job_fix_list.csv", encoding='cp949') as f:
        urls = f.readlines()

#     url_list = []

#     for url in urls:
#         url_spl = url.rstrip("\n").split(",")

#         job = ",".join(url_spl[:-1]).strip('""')
#         url = url_spl[-1]

#         url_list.append([job, url])

    url_list = [url.rstrip("\n") for url in urls]
    
    return url_list

def get_job_description(driver, url):
    
    # Job Description 을 담을 딕셔너리 생성
    res = {}
    
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            
            # url 접속하기
            driver.get(url)

            # 화면 비율을 축소해서 모든 데이터 보이게 만들기
            driver.execute_script("document.body.style.zoom='5%'")

            time.sleep(5)

            # url 에서 index 값 추출
            res['id'] = url.split("/")[-1]

            # JobHeader 클래스 가져와서 title, company, tags 추출
            job_header = driver.find_element(By.CLASS_NAME, 'JobHeader_className__HttDA')
            title = job_header.find_element(By.TAG_NAME, 'h2')
            company = job_header.find_element(By.TAG_NAME, 'a')
            location = job_header.find_element(By.CLASS_NAME, 'JobHeader_pcLocationContainer__xRwIv')
            tags = job_header.find_element(By.CLASS_NAME, 'Tags_tagsClass__mvehZ')

            # res 딕셔너리에 저장
            res['title'] = title.text
            res['company'] = company.text
            res['location'] = location.text.split(".")

            # Tag 리스트 가져와서 저장
            tag_list = list(map(str, tags.text.split('#')))
            tag_list = tag_list[1:]
            res['tags'] = tag_list

            # JobDescription 클래스 가져와서 header, contents 추출
            jd = driver.find_element(By.CLASS_NAME, 'JobDescription_JobDescription__VWfcb')
            jd_header = jd.find_elements(By.TAG_NAME, 'h6')
            jd_contents = jd.find_elements(By.TAG_NAME, 'p')

            for i in range(1, len(jd_header)+1):
                title = jd_header[i-1].text
                content = jd_contents[i].text.split("\n")

                if title == '자격요건':
                    res['qualifications'] = content
                elif title == '우대사항':
                    res['preferred'] = content
                elif title == '혜택 및 복지':
                    res["benefits"] = content
                elif title == '기술스택 ・ 툴':
                    res["skills"] = content


            jwp = driver.find_element(By.CLASS_NAME, "JobWorkPlace_className__ra6rp")
            jwp_bodies = jwp.find_elements(By.CLASS_NAME, 'body')

            if len(jwp_bodies) < 2:
                workplace = ""
            else:
                workplace = jwp_bodies[1].text

            res['workplace'] = workplace
        except NoSuchElementException:
            print("존재하지 않는 페이지 입니다.")
    else:
        print("존재하지 않는 페이지 입니다.")
        print("res = ", res)
    
    return res

# 크롤링 실행

In [20]:
driver = webdriver.Chrome()

# url 가져오기
url_list = get_url_list()

total = []

for i in tqdm(range(3000, 4000)):
    res = get_job_description(driver, url_list[i])
    
    if len(res) == 0:
        print("존재하지 않는 페이지 입니다.")
        continue
    else:
        total.append(res)

driver.quit()

 10%|▉         | 97/1000 [12:08<2:04:22,  8.26s/it]

존재하지 않는 페이지 입니다.


 37%|███▋      | 371/1000 [43:59<50:35,  4.83s/it]  

존재하지 않는 페이지 입니다.
res =  {}
존재하지 않는 페이지 입니다.


 45%|████▍     | 447/1000 [52:40<46:01,  4.99s/it]  

...

 83%|████████▎ | 832/1000 [1:37:11<21:24,  7.64s/it]

존재하지 않는 페이지 입니다.


 92%|█████████▏| 916/1000 [1:46:34<06:45,  4.83s/it]

존재하지 않는 페이지 입니다.
res =  {}
존재하지 않는 페이지 입니다.


100%|██████████| 1000/1000 [1:56:17<00:00,  6.98s/it]


In [ ]:
url_list = get_url_list()

print(len(url_list))

5462


# 크롤링 결과 및 확인

In [21]:
print(total[0])
for t in total:
    print(t)


{'id': '135906', 'title': '모바일 앱 개발자(Flutter)', 'company': '코멘토', 'location': ['서울', '한국'], 'tags': ['인원급성장', '50명이하', '설립4~9년', '출산휴가', '커피', '간식', '위워크', '세미나참가비', '교육'], 'qualifications': ['• Flutter를 이용해 Logic/UI 구현이 가능하신 분', '• Bloc/Provider 등의 상태관리 라이브러리 이해가 있으신 분', '• LEAN하게 팀과 일할 수 있으신 분', '• ENJOY함을 가지고 계신 분', '• 본인의 꿈이 있으신 분', '• 남성의 경우 군필 또는 병역면제자', '* 학력 무관'], 'preferred': ['• 코멘토가 해결하려고 하는 진로 문제의 해결, 기회와 정보의 평등에 대한 관심이 있으신 분', '• 앱 개발에서 출시까지 한 사이클을 경험해보신 분', '• CI/CD 파이프라인을 구축해보신 분', '• WebView UI와 네이티브 UI를 연동해보신 분', '• 코드 개선을 위한 어플리케이션 구조 설계, 코드리뷰를 해보신 분'], 'benefits': ['1. 개인 장비 : 업무용 노트북 제공(개발자는 맥북 제공), 모니터 제공, 개인 필요 장비 지원', '2. 자기 계발 : 조직문화활동, 도서구입비, 세미나 참가비 지원', '3. 통근, 교통 : 위워크 서울스퀘어 입주', '4. 식사, 간식 : 점심식대 제공, 야근식대 제공, 프리미엄 커피 무제한', '5. 연차, 휴가 : 보건휴가, 출산휴가, 법정휴가', '6. 근무형태 : 점심시간 1시간 30분', '7. 보험 의료 : 함께 울고 웃을 수 있는 경조사비 지원, 4대 보험'], 'skills': ['Git', 'Github', 'VueJS', 'CSS', 'HTML', 'JavaScript', 'PHP', 'AWS', 'Docker', 'Laravel', 'React.js', 'Flutter', 'Next.js'], 

# 결과 csv 파일로 저장하기

In [16]:
import csv

with open('C:\codes\playground-data\crawler\output/wanted_crawling_output_8_3001_4000.csv', 'w', encoding='UTF-8') as f:
# with open('C:\codes\playground-data\crawler\output/EXAMPLE.csv', 'w', encoding='UTF-8') as f:
    # csv 파일의 헤더 설정
    header = ['id', 'title', 'company', 'location', 'tags', 'qualifications', 'preferred', 'benefits', 'skills', 'workplace']

    # 헤더 적용
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    
    # 헤더에 맞게 딕셔너리 값 입력
    for jd in total:
        writer.writerow(jd)

# 전처리 (진행중)

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


exception_list = ['를', '를', '를', '을', '을', '있거나', '있으신', '갖추신', '경험이', '이상의', '에', '에', '에', '에', '에', '이용한', '에', '에', '에', '준하는', '에', '있으신', '에', '에', '에', '중심적인', '에', '에', '이해를', '에', '대한', '및', '가지고', '전반적인', '에']

# 텍스트 정의
contents = "• 6년 이상의 백엔드 개발 경력이 있거나 그에 준하는 실력을 갖추신 분\n• JavaScript / TypeScript 를 이용한 개발에 능숙하신 분\n• 웹 서비스에 대한 전반적인 이해를 갖추신 분\n• RDB(Postgres) 및 ORM에 대한 전반적인 이해를 갖추신 분\n• 도커, 쿠버네티스 등 컨테이너 기반의 서비스에 대한 경험이 있으신 분\n• 1년 이상의 NestJS 실무 프로젝트 개발 경험이 있으신 분\n• 모듈화, 테스트, 유지보수가 용이한 코드에 대한 이해를 갖추고 계신 분\n• 기술 중심이 아닌 유저 중심적인 태도를 가지고 계신 분\n• 배움에 대한 열정을 갖고, 지속적이고 효과적으로 피드백을 주고 받는 분"

# 특수문자 제거
contents = re.sub(r'[^\w\s]', '', contents)

# 단어 토큰화
tokens = word_tokenize(contents)

# 품사 태깅
tagged_words = pos_tag(tokens)

# 명사 추출 (동사 및 불용어 제외)
nouns = [word for word, pos in tagged_words if pos.startswith('N') and not pos.startswith('V') and word.lower() not in exception_list]

# 중복 제거
nouns = list(set(nouns))

# 결과 출력
print(nouns)